In [70]:
# !pip install pandas
# !pip install odfpy
import pandas as pd
import numpy as np
import matplotlib
from decimal import Decimal
from numpy import linalg as LA

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

def simpleFitting(x, y, polinom_size):
        vandermond_tr = np.zeros((polinom_size,x.size))
        
        for i in range(polinom_size):
              vandermond_tr[i] = x**i
        
        vandermond = vandermond_tr.transpose()
        
        XX = vandermond_tr.dot(vandermond)
        XY = vandermond_tr.dot(y)

        coeff_vector = LA.solve(A, XY)
        
        return coeff_vector
    
    
def GaussMethod(x, y, xo, yo, dyo, polinom_size, constr_number):
    
    vandermond_tr = np.zeros((polinom_size,x.size))
    lagrange_multypliers_matrix = np.zeros((constr_number, polinom_size))
    init_conditial_matrix = np.zeros((constr_number, polinom_size))
    function_vec = np.zeros(polinom_size)
    derivative_vec = np.zeros(polinom_size)
    
    for i in range(polinom_size):
        function_vec[i] = xo**i
        if xo!=0:
            derivative_vec[i] = i * xo**(i-1)
        vandermond_tr[i] = x**i

############################################### Fill initial conditional and Lagrange matrixes
    for j in range(constr_number):
        init_conditial_matrix[j] = function_vec if j==0 else derivative_vec
        lagrange_multypliers_matrix[j] = -0.5 * ( function_vec if j==0 else derivative_vec )


 ####################### Simple matrix fitting   

    vandermond = vandermond_tr.transpose()        
    XX = vandermond_tr.dot(vandermond)
    XY = vandermond_tr.dot(y)
    XY = np.append(XY, yo)
    XY = np.append(XY, dyo) if constr_number == 2 else XY


##################### Create matrix for Algebraic system solving

    lagrange_multypliers_matrix = lagrange_multypliers_matrix.transpose()
    init_conditial_matrix = np.hstack(( init_conditial_matrix, np.zeros((constr_number,constr_number)) ))
    
    A = np.hstack((XX,lagrange_multypliers_matrix))
    A = np.vstack((A, init_conditial_matrix))

#     print(lagrange_multypliers_matrix)
#     print("------")
#     print(A)


    coeff_vector = LA.solve(A, XY)
    ### to remove Lagrange variables
    coeff_vector = coeff_vector[:polinom_size]
#     print ("coeff_vector", coeff_vector)
    return coeff_vector


###################################### FUNCTION CURVE
def calculateCurveValue(xo, coeff_vector, coeff_number):
    # print("coeff_vector", coeff_vector)
    value = 0
    
    for n in range(coeff_number):
        value += coeff_vector[n] * xo**n

    return value
########################################### DERIVATIVE CURVE

def calculateDerivative(xo, coeff_vector, coeff_number):
    derivative = 0
    
    for n in range(1, coeff_number):
        derivative += n* coeff_vector[n] * xo**(n-1)
        
    return derivative

########################################### SECOND DERIVATIVE CURVE

def calculateSecondDerivative(x, coeff_vector):
    second_derivative = 0

    for n in range(2, coeff_number):
        second_derivative += n* (n - 1) * coeff_vector[n] * xo**(n-2)
        
    return second_derivative
###############################################################

In [71]:
###################################################
pd.set_option("display.max.columns", None)
df = pd.read_excel("Dimin_sample.ods", sheet_name='ceramic1_RPP1')

#######
excel_data = df.to_numpy()
x=excel_data[:,0]
y=excel_data[:,1]

######################################
spline_number =1
coeff_number = 4
discrete_size= 1000
step = 1./discrete_size
x_zero_value = 0
x_one_value = 1

begin = 1
end = np.min(x) * 0.99
# end = 0
x_board = np.linspace(begin, end, spline_number+1)

#######################################################
last_spline_number =1
last_coeff_number = 4
last_x_board = np.linspace(x_board[-2], end, last_spline_number+1)

print ("x_board", x_board)
print ("last_x_board", last_x_board)
if (spline_number ==1):
    X = [x[np.logical_and(x_board[i-1 ] > x, x >= x_board[i])] for i in range(1, x_board.size)]
    Y = [y[np.logical_and(x_board[i-1] > x, x >= x_board[i])] for i in range(1, x_board.size)]
else:
######################################################
    index_first = np.logical_and(x_board[0] > x, x >= (x_board[1] + x_board[2])/2 )
    index_last = np.logical_and((x_board[x_board.size - 2] + x_board[x_board.size - 3])/2 > x, x >= x_board[x_board.size - 1] )

    indexes = [np.logical_and((x_board[i - 1] + x_board[i])/2 > x, x >= (x_board[i+1] + x_board[i+2])/2 ) for i in range(1, x_board.size - 2)]
    indexes.insert(0, index_first)
    indexes.append(index_last)

    X = [x[ind] for ind in indexes]
    Y = [y[ind] for ind in indexes]

last_X = [x[np.logical_and(last_x_board[i-1 ] > x, x >= last_x_board[i])] for i in range(1, last_x_board.size)]
last_Y = [y[np.logical_and(last_x_board[i-1] > x, x >= last_x_board[i])] for i in range(1, last_x_board.size)]

#     index_first = np.logical_and(last_x_board[0] > x, x >= (last_x_board[1] + last_x_board[2])/2 )
#     index_last = np.logical_and((last_x_board[last_x_board.size - 2] + last_x_board[last_x_board.size - 3])/2 > x, x >= last_x_board[last_x_board.size - 1] )

#     indexes = [np.logical_and((last_x_board[i - 1] + last_x_board[i])/2 > x, x >= (last_x_board[i+1] + last_x_board[i+2])/2 ) for i in range(1, last_x_board.size - 2)]
#     indexes.insert(0, index_first)
#     indexes.append(index_last)

#     last_X = [x[ind] for ind in indexes]
#     last_Y = [y[ind] for ind in indexes]
    
######################################################
# print (len(X))
print("-----------------------")
print (last_Y)
# print (np.min(x), np.max(x))

coeff_matrix =np.zeros((spline_number ,coeff_number), np.float64)
# last_coeff_matrix =np.zeros((last_spline_number,last_coeff_number))

splines = np.zeros((spline_number, discrete_size ))
# last_splines = np.zeros((last_spline_number, discrete_size), np.float64)
# first_derivative = np.zeros((spline_number, discrete_size))
# second_derivative = np.zeros((spline_number, discrete_size))

x_board [1.0000000e+00 2.7867213e-04]
last_x_board [1.0000000e+00 2.7867213e-04]
-----------------------
[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00243789, 0.00466109, 0.00808734, 0.0139313 ,
       0.0203678 , 0.0203689 , 0.0330994 , 0.0505512 , 0.0778607 ,
       0.123638  , 0.166678  , 0.266763  , 0.363963  , 0.381067  ,
       0.491254  , 0.628727  , 0.736875  , 0.779641  , 0.895608  ,
       0.910857  , 0.954262  , 0.977572  , 0.986949  , 0.991642  ,
       0.99451   , 0.996363  , 0.997552  , 0.999629  ])]


In [72]:
#############################################CALCULATE COEFFICIENTS MATRIX

yo =0
dyo=0


for i in range(spline_number - 1):
    if i == 0:
        coeff_matrix[i, :] = GaussMethod(X[i], Y[i], x_one_value, 1, 0, coeff_number, 1)
    else:
        coeff_matrix[i, :] = GaussMethod(X[i], Y[i], x_board[i], yo, dyo, coeff_number, 2)

    yo = calculateCurveValue(x_board[i + 1], coeff_matrix[i, :], coeff_number);
    dyo = calculateDerivative(x_board[i + 1], coeff_matrix[i, :], coeff_number);

######## CALCULATE LAST COEFF MATRIX
for i in range(last_spline_number):
    if i == last_spline_number - 1:
        last_coeff_matrix[i, :] = GaussMethod(last_X[i], last_Y[i], x_zero_value, 0, 0, last_coeff_number, 1)
    else:
        last_coeff_matrix[i, :] = GaussMethod(last_X[i], last_Y[i], last_x_board[i], yo, dyo, last_coeff_number, 2)
        
    yo = calculateCurveValue(last_x_board[i + 1], last_coeff_matrix[i, :], last_coeff_number);
    dyo = calculateDerivative(last_x_board[i + 1], last_coeff_matrix[i, :], last_coeff_number);

######################################## CREATE SPLINES 
temp = np.linspace(begin, x_board[-2], splines.size)

for i in range(spline_number - 1):
    X_temp = temp[np.logical_and(temp <= x_board[i], temp >= x_board[i+1])]
    splines[i] = calculateCurveValue(X_temp, coeff_matrix[i, :], coeff_number)
        
last_temp = np.linspace(last_x_board[0], last_x_board[-1], last_splines.size)
for i in range(last_spline_number):
        X_temp = last_temp[np.logical_and(last_temp <= last_x_board[i], last_temp >= last_x_board[i+1])]
#         print(X_temp.size)
        last_splines[i] = calculateCurveValue(X_temp, last_coeff_matrix[i, :], last_coeff_number)       

    
#     first_derivative[i] = calculateDerivative(X_temp, coeff_matrix[i, :], coeff_number)
#     second_derivative[i] = calculateSecondDerivative(X_temp, coeff_matrix[i, :])


NameError: name 'A' is not defined

In [ ]:
################################################ MAKE PLOTS
splines = splines.reshape(splines.size)
# last_splines = last_splines.reshape(last_splines.size)


plt.plot(temp, splines, linewidth=2)
# plt.plot(last_temp, last_splines, linewidth=2)
# plt.plot(extr_temp, extr_spline, "r", linewidth=2)
plt.plot(x, y, 'bo')
# plt.plot(X[0], Y[0], 'bo')
# plt.plot(X[1], Y[1], 'ro')
# plt.plot(X[2], Y[2], 'go')
# plt.plot(X[3], Y[3], 'ko')
# plt.plot(X[4], Y[4], 'yo')

plt.grid(True)
plt.show()


In [ ]:
# print(coeff_matrix)
# print('%.17E' % Decimal('408.03') )
# num = 408.03
# # coeff_matrix = coeff_matrix.astype(float64)
# print(num)
# print('{:.17e}'.format(float(num)))

# print('{:.17e}'.format(float(c)))



########## CREATE FILE
# print("extrapolation coeefficients",c, a)
# print("last_x_board",last_x_board)
# print("x_board",x_board)
# print("last coefficient matrix\n" , last_coeff_matrix)
# print("coefficient matrix\n" , coeff_matrix)
# with open("Arseniys_curve_coefficients.txt", "a+") as f:
#     np.savetxt(f, np.array(([12]) ) )
#     np.savetxt(f, np.array(([3]) ) )
#     np.savetxt(f, np.array(([0, xo]) ) )
#     np.savetxt(f, np.array(([c, 0, a]) ) )
#     for i in range (last_x_board.size - 1):
# #         f.write("------------------\n")
#         np.savetxt(f, np.array(( [last_coeff_number] ) ) )
#         np.savetxt(f, np.array(([last_x_board[-1-i], last_x_board[-2-i] ]) ) )
#         np.savetxt(f, last_coeff_matrix[-1 - i])
# #     f.write("------------------\n")
# #     f.write("------------------\n")
#     for i in range (0, x_board.size - 2):
# #         f.write("------------------\n")
#         np.savetxt(f, np.array(( [coeff_number] ) ) )
#         np.savetxt(f, np.array(([x_board[-2-i], x_board[-3-i] ]) ) )
#         np.savetxt(f, coeff_matrix[-1 - i])
